In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import copy
import datetime
import pickle
from abc import abstractmethod
%matplotlib inline

In [2]:
ef = pd.ExcelFile('data/final_data_HHD.xlsx')
hs300 = ef.parse()
ef.close()
with open('../myAstock/data/two_year_panel.pickle', 'rb') as f:
    # two years stock data
    # type: Panel
    two_year = pickle.load(f)

two_year.set_axis(1, two_year.iloc[0].index.to_datetime())

## 基本要求

data当中的列的定义,及其格式应当有一个标准

以下为基本变量表格

|open    |high     |low       |close     |turnover    |Vol      | Change | Chg  |turnoverP |
|:------:|:-------:|:--------:|:--------:|:----------:|:-------:|:------:|:-----|:--------:|
| 开盘价  |最高价    | 最低价     | 收盘价    |换手率      | 成交量    |涨跌额   | 涨跌幅|成交额     |

## Strategy

类内容:
- \_\_init\_\_ 当中决定要操作的股票, 以及最大历史数据长度(默认30), 交易频度(默认一天)
- 多种买卖操作
- 在handle_data(data, context)之内进行买卖判断操作
 - data当中有过去一年的所有数据
 - context当中有:
    - 用户当前所剩资金(用户数据)
    - 已经投资的资金(用户数据)
    - 所投资的股票,买入时间,及其盈亏(用户数据)


之后在Loopbacker之内进行回测操作

In [3]:
from loopback import *

## Loopbacker 

类功能:
- 读取所有股市数据
- 使用一个Strategy类进行回测
 - Strategy中handler的data与context的定义,及该函数的调用
 - 由Strategy当中的初始化操作进行选股

In [27]:
    stocks = copy.copy(two_year)
    stocks['000300'] = hs300
    calenda = list(hs300.index)
    TOTAL_MONEY = 5e6

In [33]:
p = stocks.transpose(1,0,2)

In [37]:
p[p.keys()[0]]

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
000001,10.240,10.330,10.300,10.170,487782.16,0.050,0.49,10.226,10.308,10.466,460958.07,508212.43,562219.45,0.50
000002,9.120,9.190,9.170,9.100,436507.16,0.050,0.55,9.126,9.208,9.361,389202.75,555152.26,604620.97,0.45
000004,14.550,14.950,14.920,14.510,29985.06,0.410,2.83,14.564,14.723,14.778,16843.93,21286.82,25419.07,3.57
000005,2.570,2.640,2.640,2.570,132198.19,0.060,2.33,2.590,2.607,2.595,131281.25,160231.70,165228.86,1.45
000006,5.420,5.440,5.420,5.350,100227.45,0.050,0.93,5.330,5.376,5.212,135352.16,223053.35,231815.22,0.75
000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000009,13.100,13.550,13.270,12.880,704693.44,0.200,1.53,13.168,12.728,11.349,872450.34,961376.03,529273.88,4.73
000010,7.410,7.490,7.430,7.350,76195.14,0.040,0.54,7.336,7.314,7.104,87560.75,138221.26,116772.94,2.13
000011,7.890,7.940,7.920,7.810,34872.48,0.060,0.76,7.984,7.938,7.610,93730.94,109293.10,92478.60,1.98


In [34]:
p.iloc[0]

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
000001,10.240,10.330,10.300,10.170,487782.16,0.050,0.49,10.226,10.308,10.466,460958.07,508212.43,562219.45,0.50
000002,9.120,9.190,9.170,9.100,436507.16,0.050,0.55,9.126,9.208,9.361,389202.75,555152.26,604620.97,0.45
000004,14.550,14.950,14.920,14.510,29985.06,0.410,2.83,14.564,14.723,14.778,16843.93,21286.82,25419.07,3.57
000005,2.570,2.640,2.640,2.570,132198.19,0.060,2.33,2.590,2.607,2.595,131281.25,160231.70,165228.86,1.45
000006,5.420,5.440,5.420,5.350,100227.45,0.050,0.93,5.330,5.376,5.212,135352.16,223053.35,231815.22,0.75
000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000009,13.100,13.550,13.270,12.880,704693.44,0.200,1.53,13.168,12.728,11.349,872450.34,961376.03,529273.88,4.73
000010,7.410,7.490,7.430,7.350,76195.14,0.040,0.54,7.336,7.314,7.104,87560.75,138221.26,116772.94,2.13
000011,7.890,7.940,7.920,7.810,34872.48,0.060,0.76,7.984,7.938,7.610,93730.94,109293.10,92478.60,1.98


In [29]:
stocks.get('000001',datetime.datetime(2014,9,1),'close')

TypeError: get() takes from 2 to 3 positional arguments but 4 were given

In [4]:
my_s = Strategy(

In [5]:
my_l = Loopbacker(my_s)

In [6]:
my_l.start()

In [19]:
'dfd' in my_l.user.state.index

True

In [38]:
s = my_l.user.state.ix['dfd']

In [42]:
my_l.user.state

,总额,盈亏,仓库,现金
2014-10-08 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-09 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-10 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-13 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-14 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-15 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-16 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-17 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-20 00:00:00,5000000.0,0.0,0.0,5000000.0
2014-10-21 00:00:00,5000000.0,0.0,0.0,5000000.0


In [41]:
s.总额 = 6

In [24]:
class B(BaseException):
    pass

In [26]:
raise B("not")

B: not